In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataloader import make_dataloaders
from model import ResNet
from model import ConvNet
from model import SE_ResNet

In [2]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on CPU.


In [7]:
model_list = os.listdir("../Models")
print(model_list)

['.ipynb_checkpoints', '09-06-20:1456_ResNet_lr=0.01_droprate=0.6_blocks=1_features=32_height=128', '09-06-20:1046_ConvNet_lr=0.01_droprate=0.6_blocks=0_features=64_height=128']


In [16]:
#Define hyperparameters and model
batch_size = 256 
num_epochs = 10
model_choice = "ConvNet" #"SE_ResNet" "ResNet" "ConvNet"
n_features = 64
height = 128
width = 256
droprate = 0.6
lr = 0.01
num_blocks = 0
r = 16
    
train_loader,test_loader = make_dataloaders(height, width, batch_size)

#initialize model and sent to device
if model_choice == "SE_ResNet":
    model = SE_ResNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate, num_blocks=num_blocks, r=r).float()
    model.to(device)
    print("SE_ResNet initialized")
    
elif model_choice == "ResNet":
    model = ResNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate, num_blocks=num_blocks).float()
    model.to(device)
    print("ResNet initialized")
    
elif model_choice == "ConvNet":
    model = ConvNet(n_in=7, n_features=n_features, height=height, width=width, droprate=droprate).float()
    model.to(device)
    print("ConvNet initialized")

else:
    sys.exit("The chosen model isn't valid")

ConvNet initialized


In [17]:
model.load_state_dict(torch.load("../Models/{}".format(model_list[2]),map_location=device))
model.eval()

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(7, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.6, inplace=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.6, inplace=False)
    (8): ReLU()
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Dropout(p=0.6, inplace=False)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): BatchNorm2d(256, eps=1e-05, mo

In [18]:
for data, target,scaler in test_loader:
    data, scaler = data.to(device), scaler.to(device)
    with torch.no_grad():
        output = model(data, scaler)
    predicted = output.argmax(1).cpu()

    test_correct += (target == predicted).sum().item()

    c = (predicted == target).squeeze()
    for i in range(data.shape[0]):
        label = target[i]
        class_correct[label] += c[i].item()
        class_total[label] += 1

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1073741824 bytes. Error code 12 (Cannot allocate memory)
